# <font color='red'><center>Code Source Projet Analyse de Données</center></font>


## <center>Prétraitement des données</center>

### <font color='blue'>Lecture des données</font>

In [ ]:
#librairies à importer
import numpy as np
import pandas as pd
import plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot
import matplotlib.pyplot as plt

#Lecture des données
X = pd.read_csv("timesData.csv")
X

### <font color='blue'>Nettoyage des données</font>

In [ ]:
#Les informations sur les variables avant le nettoyage
def info():
    inf = pd.DataFrame(index=X.columns)
    inf['null_count'] = X.isnull().sum()
    inf['unique_count'] = X.nunique()
    inf['data_type'] = X.dtypes

    return inf
info()

In [25]:
#Suppression des colonnes non désirées
to_drop=['student_staff_ratio','international_students','female_male_ratio']
X.drop(to_drop, inplace=True, axis=1)

In [26]:
df=X[X['year']>=2016]


In [27]:
#Suppression des valeurs nulles
df=df.dropna()

In [28]:
#Convertir les variables num_students et income en variable numérique
df['num_students'] = df.num_students.apply(lambda x: x.replace(',', '.'))
df['num_students'] = pd.to_numeric(df['num_students'])
df['income'] = df.income.apply(lambda x: x.strip('-'))
df['income'] = pd.to_numeric(df['income'])


In [29]:
df=df.head(150)

In [30]:
#Arrangement de la variable world_rank
df["world_rank"]=range(1,151)


In [ ]:
df

In [ ]:
#Les informations sur les variable après nettoyage
def info():
    inf = pd.DataFrame(index=df.columns)
    inf['null_count'] = df.isnull().sum()
    inf['unique_count'] = df.nunique()
    inf['data_type'] = df.dtypes

    return inf
info()

### <font color='blue'>Visualisation des données</font>

In [ ]:
#Diagramme à bandes : Citations, enseignement et recherche des 3 meilleures universités en 2016 

df1 = df.iloc[:3,:]
x = df1.university_name

trace1 = {
  'x': x,
  'y': df1.citations,
  'name': 'citation',
  'type': 'bar',
  'marker':dict(color='#770737'),
  'text': df1.citations,
};
trace2 = {
  'x': x,
  'y': df1.teaching,
  'name': 'teaching',
  'type': 'bar',
  'marker':dict(color="#9f5173"),
  'text': df1.teaching,
};
trace3 = {
    'x':x,
    'y':df1.research,
    'name':'research',
    'marker':dict(color='#d6b4c3'),
      'type':"bar",
    'text': df1.research
}
data = [trace1, trace2, trace3];
layout = {
  'xaxis': {'title': 'Top 3 universities'},
  'barmode': 'relative',
  'title': 'citations, teaching and research of top 3 universities in 2016',
  'template' : 'plotly_white'
};
fig = go.Figure(data = data, layout = layout)
fig.update_traces(textposition='inside')
fig


In [ ]:
#Diagramme circulaire : Taux des étudiants des 10 meilleures universités en 2016
#installation de plotly est important

df2 = df.iloc[:10,:]
pie_list = [
    float(val) 
    for val in df2.num_students
] 
# another mehtod to directly create a figure
fig = {
  "data": [
    {
      "values": pie_list,
      "labels": df2.university_name,
      "hoverinfo":"label+percent+name",
      "hole": .3,
      "type": "pie"
    },],
  "layout": {
        "title":"Students rates of top 10 universities in 2016",
        "annotations": [
            { "font": { "size": 15},
              "showarrow": False,
              "text": "Number of Students",
            },
        ]
    }
}
iplot(fig)

In [ ]:
# Nuage des mots des pays cités en 2016
#installation de librairie WorldCould
from wordcloud import WordCloud
df3 = df.country
plt.subplots(figsize=(10,10))
wordcloud = WordCloud(background_color='white',).generate(" ".join(df3))
plt.imshow(wordcloud)
plt.axis('off')
plt.show()

## <center>Analyse des données</center>

## <font color='blue'>Méthode de régression linéaire</font>

In [ ]:
#On cherche s'il existe une relation entre research et teaching
from numpy.random import randn
X = df['research'].values
Y = df['teaching'].values

# Affichage du nuage de points
plt.title("Research vs Teaching (Testing set)")
plt.xlabel("research")
plt.ylabel("Teaching")
plt.scatter(X,Y)


In [39]:
#Coefficient de corrélation
from scipy.stats import pearsonr
coeff_pearson,_ = pearsonr(X,Y)
print("coefficient de correlation = {}".format(coeff_pearson))

coefficient de correlation = 0.8811320101289077


In [ ]:
#Application de la régression linéaire

from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
lm = LinearRegression()
X = df[['research']].values
Y = df['teaching'].values
lm.fit(X,Y)
Ypred=lm.predict(X)
plt.title("Research vs Teaching (Testing set)")
plt.xlabel("research")
plt.ylabel("Teaching")

print("l'intercept est: ",lm.intercept_)
print("la pente est: ",lm.coef_)

#Affichage de nuage des points et de la ligne de régression
plt.plot(X, Y,'o')
plt.plot(X, Ypred, color = "red")


## <font color='blue'>Méthode ACP</font>

In [ ]:
#installation de package fanalysis
#importer la classe de calcul
from fanalysis.pca import PCA

#Data frame d'ACP contient les variables numériques
newdata = {
    "teaching" : df['teaching'], 
    "research" : df['research'],
    "citations" : df['citations'],
    "income" : df['income'],
}
df_acp = pd.DataFrame(newdata)

#nombre d'observations
df_acp.shape


In [ ]:
#Matrice de corrélation
df_acp.corr()

In [ ]:
#instancier l'objet de calcul
my_pca = PCA(std_unit=True, row_labels=df_acp.index.values , col_labels=df_acp.columns.values)

#Lancement des calculs
X1 = df_acp.to_numpy()
my_pca.fit(X1)


##### Analyse des valeurs propres


In [ ]:
#Les valeurs propres
print(my_pca.eig_)


In [ ]:
#les valeurs propres en valeur absolue
my_pca.plot_eigenvalues()

In [ ]:
#les valeurs propres en pourcentage de la variance totale
my_pca.plot_eigenvalues(type="percentage")


In [ ]:
#les valeurs propres en pourcentage cumulé de la variance totale
my_pca.plot_eigenvalues(type="cumulative")


##### Extraction des statistiques sur les points colonnes


In [ ]:
#statistiques pour les points colonnes
df_cols = my_pca.col_topandas()
df_cols

##### Extraction des statistiques sur les points lignes


In [ ]:
#statistiques pour les points colonnes
df_rows = my_pca.row_topandas()
df_rows

In [ ]:
# Classement des points lignes en fonction de leur contribution au 1er axe
my_pca.plot_row_contrib(num_axis=1, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur cos2 sur le 1er axe
my_pca.plot_row_cos2(num_axis=1, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur contribution au 2eme axe
my_pca.plot_row_contrib(num_axis=2, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur cos2 sur le 2eme axe
my_pca.plot_row_cos2(num_axis=2, nb_values=10)


##### Graphiques

In [ ]:
#cercle des corrélations
my_pca.correlation_circle(num_x_axis=1, num_y_axis=2)


In [ ]:
#Graphe des individus
my_pca.mapping_row(num_x_axis=1, num_y_axis=2,figsize=(13,13))


In [ ]:
#Graphe des individus et des variables
my_pca.mapping(num_x_axis = 1  , num_y_axis= 2,figsize=(13,13))


## <font color='blue'>Méthode AFC</font>

In [125]:
#Data frame d'AFC contient des variables qualitatives
afc_pd = {
    'country' : df['country'], 
    'worldrank' : df['world_rank']
}

df_afc = pd.DataFrame(afc_pd)

In [ ]:
#convert la variable world_rank a une variable qualitatives 
#si world_rank est inférieur à 50 donc c'est classement Top
#si world_rank est compris entre 50 et 100 donc c'est classement Middle
#si world_rank est supérieus à 100 donc c'est classement Low level

from pandas._libs import index
from matplotlib.cbook import index_of
for r in df_afc.index : 
  if df_afc.worldrank[r] <= 50 : 
    df_afc.worldrank[r] = 'Top'
  elif df_afc.worldrank[r] > 50 and df_afc.worldrank[r] < 100 :
    df_afc.worldrank[r] = 'Middle' 
  elif df_afc.worldrank[r] >=  100 :
    df_afc.worldrank[r] = 'Low level' 

df_afc


In [ ]:
#table des individus et variables
data_crosstab = pd.crosstab(df_afc['country'], 
                            df_afc['worldrank'],
                            margins = False)
print(data_crosstab) 

In [ ]:
#importation de la librairie
from fanalysis.ca import CA

#lancer les calculs
my_afc = CA(row_labels=data_crosstab.index.values, col_labels=data_crosstab.columns.values)
X1 = data_crosstab.to_numpy()
my_afc.fit(X1)


##### Analyse des valeurs propres


In [ ]:
#affichage des valeurs propres
print(my_afc.eig_)


In [ ]:
#affichage graphique des v.p. 
my_afc.plot_eigenvalues()

In [ ]:
#les valeurs propres en pourcentage de la variance totale
my_afc.plot_eigenvalues(type="percentage")

In [ ]:
#les valeurs propres en pourcentage cumulé de la variance totale
my_afc.plot_eigenvalues(type="cumulative")


##### Extraction des statistiques sur les points lignes


In [ ]:
#informations sur les modalités lignes
print(my_afc.row_topandas())

In [ ]:
# Classement des points lignes en fonction de leur cos2 sur le 1er axe
my_afc.plot_row_cos2(num_axis=1, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur contribution au 1er axe
my_afc.plot_row_contrib(num_axis=1, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur cos2 sur le 2ème axe
my_afc.plot_row_cos2(num_axis=2, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur contribution au 2ème axe
my_afc.plot_row_contrib(num_axis=2, nb_values=10)


##### Graphiques

In [ ]:
# Mapping simultané des points lignes et colonnes
my_afc.mapping(num_x_axis=1,num_y_axis=2,figsize=(12,12))

In [ ]:
# Mapping des points lignes
my_afc.mapping_row(num_x_axis=1, num_y_axis=2,figsize=(8,8))

In [ ]:
# Mapping des points colonnes
my_afc.mapping_col(num_x_axis=1, num_y_axis=2,figsize=(8,8))

## <font color='blue'>Méthode ACM</font>

In [202]:
#Data frame utilisé dans la méthode ACP
acm_pd = {
    'country' : df.country , 
    'worldrank' : df.world_rank,
    'num_students' : df.num_students,

    
}
df_acm = pd.DataFrame(acm_pd)


In [ ]:
#convert la variable num_students a une variable qualitative 

for r in df_acm.index : 
  if df_acm.num_students[r] >= 30.000  : 
    df_acm.num_students[r] = 'High number'
  elif df_acm.num_students[r] < 30.000 and df_acm.num_students[r] > 10.000 :
    df_acm.num_students[r] = 'Middle number' 
  elif df_acm.num_students[r] <= 10.000   :
    df_acm.num_students[r] = 'Low number' 


In [ ]:
#convert la variable world_rank a une variable qualitative 

for r in df_acm.index : 
  if df_acm.worldrank[r] <= 50 : 
    df_acm.worldrank[r] = 'Top'
  elif df_acm.worldrank[r] > 50 and df_acm.worldrank[r] < 100 :
    df_acm.worldrank[r] = 'Middle' 
  elif df_acm.worldrank[r] >=  100 :
    df_acm.worldrank[r] = 'Low level' 



In [ ]:
data_crosstab = pd.crosstab(df_acm['country'], 
                            [df_acm['worldrank'] , df_acm['num_students'] ],  
                               margins = False)
data_crosstab

In [ ]:
from fanalysis.mca import MCA
#instanciation
my_acm = CA(row_labels=data_crosstab.index.values, col_labels=data_crosstab.columns.values)

#lancer les calculs
X3 = data_crosstab.to_numpy()
my_acm.fit(X3)


##### Analyse des valeurs propres


In [ ]:
#affichage des valeurs propres
print(my_acm.eig_)

In [ ]:
#affichage graphique des v.p. 
my_acm.plot_eigenvalues()

In [ ]:
#les valeurs propres en pourcentage de la variance totale
my_acm.plot_eigenvalues(type="percentage")


In [ ]:
#les valeurs propres en pourcentage cumulé de la variance totale
my_acm.plot_eigenvalues(type="cumulative")


##### Extraction des statistiques sur les points colonnes


In [ ]:
#informations sur les modalités colonnes
print(my_acm.col_topandas())

##### Extraction des statistiques sur les points lignes


In [ ]:
#informations sur les modalités lignes
print(my_acm.row_topandas())

In [ ]:
# Classement des points lignes en fonction de leur cos2 sur le 1er axe
my_acm.plot_row_cos2(num_axis=1, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur contrib sur le 1er axe
my_acm.plot_row_contrib(num_axis=1, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur cos2 sur le 2eme axe
my_acm.plot_row_cos2(num_axis=2, nb_values=10)


In [ ]:
# Classement des points lignes en fonction de leur contrib sur le 2eme axe
my_acm.plot_row_contrib(num_axis=2, nb_values=10)


##### Graphiques

In [ ]:
# Graphique des modatités colonnes

fig, ax = plt.subplots(figsize=(10,10))
ax.axis([-3,+3,-3,+3])
ax.plot([-3,+3],[0,0],color='blue',linestyle='--')
ax.plot([0,0],[-3,+3],color='blue',linestyle='--')
ax.set_xlabel("Dim.1 ")
ax.set_ylabel("Dim.2 ")
plt.title("Carte des modalités colonnes")

for i in range(data_crosstab.shape[1]):
    ax.text(my_acm.col_coord_[i,0],my_acm.col_coord_[i,1],data_crosstab.columns[i])
plt.show()

In [ ]:
# Graphique des modatités lignes
fig, ax = plt.subplots(figsize=(12,12))
ax.axis([-3,+3,-3,+3])
ax.plot([-3,+3],[0,0],color='red',linestyle='--')
ax.plot([0,0],[-3,+3],color='red',linestyle='--')
ax.set_xlabel("Dim.1 ")
ax.set_ylabel("Dim.2")
plt.title("Carte des modalités lignes")
for i in range(data_crosstab.shape[0]):
 ax.text(my_acm.row_coord_[i,0],my_acm.row_coord_[i,1],data_crosstab.index[i])

plt.show()

In [ ]:
#représentation simultanée
fig, ax = plt.subplots(figsize=(15,15))
ax.axis([-3,+3,-3,+3])
ax.plot([-4,+4],[0,0],color='silver',linestyle='--')
ax.plot([0,0],[-4,+4],color='silver',linestyle='--')
ax.set_xlabel("Dim.1")
ax.set_ylabel("Dim.2")
plt.title("Carte des modalités lignes et colonnes")
#modalités ligne
for i in range(data_crosstab.shape[0]):
 ax.text(my_acm.row_coord_[i,0],my_acm.row_coord_[i,1],data_crosstab.index[i],color='blue')
#modalités colonne
for i in range(data_crosstab.shape[1]):
 ax.text(my_acm.col_coord_[i,0],my_acm.col_coord_[i,1],data_crosstab.columns[i],color='red')

plt.show()